# Set-up

In [2]:
import numpy as np
import importlib
import time
import sys
from tqdm import tqdm
from scipy.special import eval_legendre, legendre, spherical_jn

import hmvec as hm # needed for the halo model
from utils.my_units import *

#importlib.reload(sys.modules['utils.power_spectrum_tools'])
from utils.power_spectrum_tools import *

# Halo model stuff

In [3]:
################### UNITS ###################
# The units from hmvec are:
# proper radius r is always in Mpc
# comoving momentum k is always in Mpc-1
# masses m are in Msolar
# rho densities are in Msolar/Mpc^3
# No h units anywhere

#%%
# Construct the Halo Model using hmvec
# Redshift, mass, and wavenumbers ranges
zMin, zMax, nZs = 0.005, 1.9, 98   #0.005, 1.9, 100  
mMin, mMax, nMs = 1.e10, 1.e16, 102 #1e10, 1e17, 100
kMin, kMax, nKs = 1.e-4, 1.e3, 1001
ms  = np.geomspace(mMin, mMax, nMs)     
zs  = np.linspace(zMin, zMax, nZs)      
ks  = np.geomspace(1e-4, 1e3, nKs)      

hcos = hm.HaloModel(zs, ks, ms=ms, mass_function='tinker', mdef='vir')

In [4]:
# Useful halo model quantities:
chis     = hcos.comoving_radial_distance(zs)
rvirs    = np.asarray([hcos.rvir(ms, zz) for zz in zs])
cs       = hcos.concentration()
Hubble   = hcos.h_of_z(zs)
nzm      = hcos.get_nzm()
biases   = hcos.get_bh()
deltav   = hcos.deltav(zs)
rhocritz = hcos.rho_critical_z(zs)
PzkLin = hcos._get_matter_power(zs, ks, nonlinear=False)

dvols  = chis**2. / Hubble
aas = aa(zs)
rs_nfw = rvirs/cs # scale radius for NFW profile
### check issue with pref in hmvec
#rhos_nfw = hm.rhoscale_nfw(ms, rvirs, cs) # scale density for NFW profile
rhos_nfw = my_rhoscale_nfw(ms, rvirs, cs) # scale density for NFW profile

In [ ]:
xMin, xMax, nXs = 1.e-3, 10, 1000

xs = np.geomspace(xMin, xMax, nXs) 
rhos = np.zeros((nZs, nMs, nXs))

rs = xs[None, None, :]*rs_nfw[:, :, None]/aas[:, None, None]
rhos = hm.rho_nfw(rs, rhos_nfw[:, :, None], rs_nfw[:, :, None])

def get_200critz(ms, cs, rhocritz, deltav):
    '''Get m200 and r200'''
    delta_rhos1 = deltav*rhocritz
    delta_rhos2 = 200.*rhocritz
    m200critz = hm.mdelta_from_mdelta(ms, cs, delta_rhos1, delta_rhos2)
    r200critz = hm.R_from_M(m200critz, rhocritz[:,None], delta=200.)
    return m200critz, r200critz

m200c, r200c = get_200critz(ms, cs, rhocritz, deltav)
Bs = get_B_rs(rs, zs, ms, m200c, r200c, rhocritz)
Bs_flat = 0.1*np.ones(rhos.shape)